## L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:

!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [33]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

* Import from the crewAI libray.


In [34]:
from crewai import Agent, Task, Crew

* As a LLM for your agents, you'll be using OpenAI's gpt-4o-mini.

**Optional Note**: crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the bottom of the notebook.


In [35]:
import os
import dotenv

dotenv.load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

## **CREATING AGENTS**

* Define your Agents, and provide them a ```role```, ```goal``` and ```backstory```.
* It has been seen that LLMs perform better when they are role playing.

### **Agent: Planner**

Note: The benefit of using multiple strings :

varname = "line 1 of text"
          "line 2 of text"
          
versus the triple quote docstring:

varname = """line 1 of text
             line 2 of text
          """
          
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [36]:
planner = Agent(
    role="Content Planner",
    goal="Lập kế hoạch nội dung hấp dẫn và chính xác về mặt thực tế trên chủ đề: {topic}",
    backstory="Bạn đang lên kế hoạch cho một bài viết blog về chủ đề: {topic}." 
    "Bạn thu thập thông tin giúp " "khán giả tìm hiểu điều gì đó " 
    "và đưa ra quyết định sáng suốt." 
    "Công việc của bạn là cơ sở để " 
    "Content Writer viết một bài về chủ đề này.",
    allow_delegation=False,
    verbose=True
)

### **Agent: Writer**

In [37]:
writer = Agent(
    role="Content writer",
    goal="Viết ý kiến ​​sâu sắc và chính xác về mặt thực tế về chủ đề: {topic}",
    backstory="Bạn đang viết " 
        "một ý kiến ​​mới về chủ đề: {topic}." 
        "Bạn viết bài dựa trên công việc của " 
        "Người lập kế hoạch nội dung, người cung cấp dàn ý " 
        "và bối cảnh liên quan về chủ đề . " 
        "Bạn làm theo các mục tiêu chính và " 
        "hướng của dàn ý do Content Planner cung cấp " 
        "Bạn cũng cung cấp những hiểu biết khách quan và khách quan " 
        "và hỗ trợ chúng bằng thông tin do Content Planner cung cấp. " 
        "Bạn thừa nhận trong phần ý kiến ​​của mình " 
        "khi tuyên bố của bạn là ý kiến ​​"
        "trái ngược với tuyên bố khách quan.",
    allow_delegation=False,
    verbose=True  
)

### **Agent: Editor**

In [38]:
editor = Agent(
    role="Editor",
    goal="Chỉnh sửa một bài đăng blog nhất định để phù hợp với phong cách viết của tổ chức.",
    backstory="Bạn là biên tập viên nhận được một bài đăng blog từ Content Writer." 
                "Mục tiêu của bạn là xem lại bài đăng blog " 
                "để đảm bảo rằng nó tuân theo các phương pháp hay nhất về báo chí," 
                "cung cấp các quan điểm cân bằng" 
                "khi đưa ra ý kiến ​​hoặc khẳng định , " 
                "và cũng tránh các chủ đề gây tranh cãi lớn " 
                "hoặc các ý kiến ​​khi có thể.",
    allow_delegation=False,
    verbose=True
)

### **Creating Tasks**

* Define your Tasks, and provide them a ```description```, ```expected_output``` and ```agent```.


### Task: Plan

In [39]:
plan = Task(
    description = (
               "1. Ưu tiên các xu hướng mới nhất, những người chơi chính," "và những tin tức đáng chú ý về {topic}.\n"
               "2. Xác định đối tượng mục tiêu, xem xét " "mối quan tâm và điểm yếu của họ.\n" 
               "3. Phát triển một bản trình bày chi tiết phác thảo nội dung bao gồm " "phần giới thiệu, các điểm chính và lời kêu gọi hành động.\n"
               "4. Bao gồm các từ khóa SEO và dữ liệu hoặc nguồn có liên quan."
               ),
    agent = planner,
    expected_output = "Tài liệu kế hoạch nội dung toàn diện "
    "với dàn ý, phân tích đối tượng," 
    "Từ khóa SEO và tài nguyên."
)

### Task: Write

In [40]:
write = Task(
    description = (
        "1. Sử dụng kế hoạch nội dung để tạo một bài đăng blog hấp dẫn về {topic}.\n" 
        "2. Kết hợp các từ khóa SEO một cách tự nhiên.\n" 
        "3. Các phần/Phụ đề được đặt tên chính xác theo cách hấp dẫn. \n" 
        "4. Đảm bảo bài đăng có cấu trúc với phần giới thiệu hấp dẫn, nội dung sâu sắc và phần kết luận tóm tắt.\n" 
        "5. Hiệu đính các lỗi ngữ pháp và phù hợp với giọng điệu của thương hiệu.\n"
    ),
    agent = writer,
    expected_output = "Một bài đăng trên blog được viết tốt " 
    "ở dạng markdown, sẵn sàng để xuất bản," 
    "mỗi phần nên có 2 hoặc 3 đoạn.",
)


### Task: Edit

In [41]:
edit = Task(
    description = (
        "Đọc lại bài đăng trên blog để tìm " 
        "lỗi ngữ pháp và sự phù hợp với tiếng nói của thương hiệu."
    ),
    agent = editor,
    expected_output = "Một bài đăng blog được viết tốt ở định dạng markdown," 
    "sẵn sàng để xuất bản," 
    "mỗi phần nên có 2 hoặc 3 đoạn văn.",
)


## **Creating the Crew**

* Create your crew of Agents
* Pass the tasks to be performed by those agents.
    * **Note**: For this simple example, the tasks will be performed sequentially (i.e they are dependent on 
    each other), so the order of the task in the list matters.
* verbose=2 allows you to see all the logs of the execution.

In [42]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose = True,
    async_execution=False,
    output_file='new-blog-post.md' # Example of output customization
)

2024-10-18 13:29:19,349 - 136082972185664 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## **Running the Crew**

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [43]:
output = crew.kickoff(inputs = {"topic": "Artificial Intelligence"})


# Agent: Content Planner
## Task: 1. Ưu tiên các xu hướng mới nhất, những người chơi chính,và những tin tức đáng chú ý về Artificial Intelligence.
2. Xác định đối tượng mục tiêu, xem xét mối quan tâm và điểm yếu của họ.
3. Phát triển một bản trình bày chi tiết phác thảo nội dung bao gồm phần giới thiệu, các điểm chính và lời kêu gọi hành động.
4. Bao gồm các từ khóa SEO và dữ liệu hoặc nguồn có liên quan.


# Agent: Content Planner
## Final Answer: 
**Content Plan for Blog Post on Artificial Intelligence**

### Title: "Khám Phá Trí Tuệ Nhân Tạo: Xu Hướng, Người Chơi Chính và Tương Lai"

---

### 1. Giới thiệu (Introduction)
- **Mục đích**: Giới thiệu về trí tuệ nhân tạo (AI) và tầm quan trọng ngày càng tăng của nó trong xã hội hiện đại.
- **Nội dung**: Nói rõ về những gì cho độc giả biết: AI là gì, và tại sao nó lại là xu hướng nóng nhất trong công nghệ hiện nay.
- **Nghĩa**: Tạo cảm hứng cho độc giả để họ muốn tìm hiểu sâu hơn về chủ đề này.

---

### 2. Đối Tượng Mục Tiêu (Target Aud

In [47]:
print(output)

# Khám Phá Trí Tuệ Nhân Tạo: Xu Hướng, Người Chơi Chính và Tương Lai

## 1. Giới thiệu

Trong thế giới công nghệ hiện đại, trí tuệ nhân tạo (AI) đã trở thành một trong những xu hướng nổi bật nhất, thu hút sự chú ý từ các chuyên gia công nghệ, doanh nhân và cả các nhà đầu tư. AI không chỉ đơn thuần là một thuật ngữ kỹ thuật, mà còn là một cuộc cách mạng mở ra những cơ hội và thách thức mới cho nhân loại. Thế nhưng, AI thực sự là gì? Làm thế nào nó có thể thay đổi cách chúng ta sống và làm việc? Trong bài viết này, chúng ta sẽ khám phá các khía cạnh của AI từ xu hướng đến những người chơi chính trong lĩnh vực này, đồng thời nêu bật những tin tức mới nhất đáng chú ý.

## 2. Đối Tượng Mục Tiêu

Điều quan trọng là xác định đối tượng lý tưởng cho bài viết này. Chúng ta hướng đến:

- **Chuyên gia trong lĩnh vực công nghệ:** Những người đã có nền tảng vững chắc và đang tìm kiếm các xu hướng mới trong nghiên cứu và phát triển AI.

- **Doanh nhân và nhà đầu tư:** Những người có nhãn quan kinh do